# Install Dependencies and Import Packages, Update mitreattack-python version for latest MITRE ATT&CK data but be wary of breaking changes

In [10]:
%%capture
!pip install mitreattack-python==3.0.6 stix2 beautifulsoup4 natsort mkdocs

In [11]:
%%capture
from mitreattack.navlayers import UsageLayerGenerator
from mitreattack.navlayers.core import Layer
from mitreattack.navlayers.core.objlink import LinkDiv
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf

import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re
from natsort import index_natsorted
import numpy as np
import zipfile
import io

pd.set_option('display.max_colwidth', None)

# Get all STIX

In [12]:
# download and parse ATT&CK STIX data
attackdata = attackToExcel.get_stix_data("enterprise-attack")
# get list of Pandas DataFrames for techniques, associated relationships, and citations
techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack")
techniques_df = techniques_data["techniques"]
procedure_df = techniques_data['procedure examples']
citations_df = techniques_data["citations"]

parsing relationships for type=technique: 100%|██████████| 17899/17899 [00:02<00:00, 7427.16it/s]


# Choose Group or Software IDs; Update Coverage URLs

In [13]:
# type in comma-separated list of MITRE Group IDs and Software IDs as string, no spaces
# make sure multipliers are all integers, no floats (incompatible with layers)

group_ids = 'G0128'
group_multiplier = [2]
software_ids = 'S0596'
software_multiplier = [1]

ids = group_ids.split(',') + software_ids.split(',')
multiplier_dict = {source_id:multiplier for (source_id, multiplier) in zip(ids, group_multiplier + software_multiplier)}
print(multiplier_dict)

#input_layers = ['custom.json']
input_layers = []

# Latest as of 10JUL2024
# number only version of Splunk Security Content used below, no 'v'
security_content_version = '4.35.0'
# copy and paste link for the "Source code (zip)" for the latest release from https://github.com/splunk/security_content/releases
splunk_raw_url = 'https://github.com/splunk/security_content/archive/refs/tags/v4.35.0.zip'
# replace version number to match above link, visit url to check
splunk_coverage_url = 'https://github.com/splunk/security_content/raw/v4.35.0/docs/mitre-map/coverage.json'
# update CAR coverage json filename based on newest version here: https://github.com/mitre-attack/car/tree/master/docs/coverage
car_coverage_url = 'https://raw.githubusercontent.com/mitre-attack/car/master/docs/coverage/splunk_analytic_coverage_01_08_2024.json'

{'G0128': 2, 'S0596': 1}


# Get Procedure Examples

In [14]:
# parse citations in a procedure (how a technique was used) and reformat as markdown hyperlink
def cite_link(citations):
    if type(citations) == float:
        return ''
    links = []
    for citation in citations:
        links.append((citation[11:-1], citations_df[citations_df['reference'].str.contains(citation[11:-1])].iloc[0]['url']))
    return links

procedure_dfs = []
for id in ids:
    procedure_dfs.append(procedure_df[procedure_df['source ID'] == id][['source ID', 'target ID', 'mapping description']].copy())
df = pd.concat(procedure_dfs)
df['citation'] = df['mapping description'].str.extractall(r'[^(]*(?P<citation>\(Citation: [^)]+\))').groupby(level=0).agg(lambda x: list(x))['citation']
df['description'] = df['mapping description'].str.replace(r'\([^\)]*?\)|\[|\]', '', regex=True)
df = df[['source ID', 'target ID', 'description', 'citation']]

citations_df = techniques_data['citations'][['reference', 'url']]

df['links'] = df['citation'].apply(cite_link)
df = df[['source ID', 'target ID', 'description', 'links']]

In [15]:
processed_layers = []

# process any imported layers
if len(input_layers) > 0:
    for layer_file in input_layers:
        filename = layer_file
        import_layer = Layer()
        import_layer.from_file(filename)
        layer_dict = import_layer.to_dict()
        layer_df = pd.DataFrame(layer_dict['techniques'])
        layer_df['Parent Technique ID'] = layer_df['techniqueID'].str.extract(r'(.{5})')

        order_df = layer_df.groupby(['Parent Technique ID'], as_index=False).sum(['score']).sort_values(['score'], ascending=False)

        layer_df = layer_df[layer_df['score']>0].dropna().set_index('Parent Technique ID')
        layer_df.reset_index(inplace=True)
        layer_df = layer_df[['techniqueID', 'comment', 'score']].copy()
        layer_df.rename(columns={'techniqueID': 'target ID', 'comment': 'description'}, inplace=True)
        layer_df['source ID'] = 'Observed Activity'
        layer_df['links'] = ''
        layer_df = layer_df[['source ID', 'target ID', 'description', 'links', 'score']]
        layer_df.drop_duplicates(subset=['target ID'], inplace=True)
        processed_layers.append(layer_df.copy())

processed_layers.append(df)
df = pd.concat(processed_layers)
df

,source ID,target ID,description,links
15482,G0128,T1102.002,ZIRCONIUM has used Dropbox for C2 allowing upload and download of files as well as execution of arbitrary commands.,"[(Google Election Threats October 2020, https://blog.google/threat-analysis-group/how-were-tackling-evolving-online-threats/), (Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"
2604,G0128,T1555.003,ZIRCONIUM has used a tool to steal credentials from installed web browsers including Microsoft Internet Explorer and Google Chrome.,"[(Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"
5675,G0128,T1140,ZIRCONIUM has used the AES256 algorithm with a SHA1 derived key to decrypt exploit code.,"[(Check Point APT31 February 2021, https://research.checkpoint.com/2021/the-story-of-jian/)]"
7029,G0128,T1583.001,ZIRCONIUM has purchased domains for use in targeted campaigns.,"[(Microsoft Targeting Elections September 2020, https://blogs.microsoft.com/on-the-issues/2020/09/10/cyberattacks-us-elections-trump-biden/)]"
10113,G0128,T1041,ZIRCONIUM has exfiltrated files via the Dropbox API C2.,"[(Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"
10943,G0128,T1567.002,ZIRCONIUM has exfiltrated stolen data to Dropbox.,"[(Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"
13318,G0128,T1068,ZIRCONIUM has exploited CVE-2017-0005 for local privilege escalation.,"[(Check Point APT31 February 2021, https://research.checkpoint.com/2021/the-story-of-jian/)]"
14180,G0128,T1105,ZIRCONIUM has used tools to download malicious files to compromised hosts.,"[(Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"
11294,G0128,T1204.001,ZIRCONIUM has used malicious links in e-mails to lure victims into downloading malware.,"[(Google Election Threats October 2020, https://blog.google/threat-analysis-group/how-were-tackling-evolving-online-threats/), (Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"
12011,G0128,T1036.004,ZIRCONIUM has created a run key named <code>Dropbox Update Setup</code> to mask a persistence mechanism for a malicious binary.,"[(Zscaler APT31 Covid-19 October 2020, https://www.zscaler.com/blogs/security-research/apt-31-leverages-covid-19-vaccine-theme-and-abuses-legitimate-online)]"


# Create Layer

In [16]:
layer_dict = {
    "name": "layer example",
    "versions" : {
        "attack": "14.1",
        "layer" : "4.5",
        "navigator": "4.9.1"
    },
    "domain": "enterprise-attack",
    "techniques": list(),
    "sorting": 3,
    "layout": {
		"layout": "flat",
		"aggregateFunction": "sum",
		"showID": False,
		"showName": True,
		"showAggregateScores": True,
		"countUnscored": False
	},
    "gradient": {
		"colors": [
			"#8ec843ff",
			"#ffe766ff",
			"#ff6666ff"
		],
		"minValue": 1,
		"maxValue": 100
	},
	"legendItems": [],
	"metadata": [],
	"links": [],
	"showTacticRowBackground": False,
	"tacticRowBackground": "#dddddd",
	"selectTechniquesAcrossTactics": True,
	"selectSubtechniquesWithParent": False
}

In [17]:
techniques_dict = {}

def df_to_layer(row, techniques_dict, techniques_df):
    # technique exists
    if row['target ID'] in techniques_dict:
        technique = dict()
        technique['techniqueID'] = row['target ID']
        all_tactics = techniques_df[techniques_df['ID']==row['target ID']].iloc[0]['tactics']
        technique['tactic'] = techniques_dict[row['target ID']]['tactic']

        technique['metadata'] = techniques_dict[row['target ID']]['metadata']
        technique['metadata'].append(
            {'name': row['source ID'], 'value': row['description']}
        )
        technique['links'] = techniques_dict[row['target ID']]['links']

        technique['links'].append({'divider': True})

        # based on ID
        if row['source ID'] in multiplier_dict:
            newScore = techniques_dict[row['target ID']]['score'] + multiplier_dict[row['source ID']]
        # imported layer, use given score
        else:
            newScore = techniques_dict[row['target ID']]['score'] + int(row['score'])
        technique['score'] = newScore

        for label, link in row['links']:
            link_dict = dict()
            link_dict['label'] = label
            link_dict['url'] = link
            technique['links'].append(link_dict.copy())
        techniques_dict[row['target ID']] = technique.copy()
    # technique does not exist
    else:
        technique = dict()
        technique['techniqueID'] = row['target ID']

        # some techniques belong to more than one tactic
        # this is a prioritization of which they will be assigned to to prevent duplicates
        tactics_priority = ['Command and Control', 'Exfiltration', 'Execution', 'Initial Access',
                            'Discovery', 'Lateral Movement', 'Defense Evasion', 'Persistence',
                            'Privilege Escalation', 'Credential Access', 'Collection', 'Impact',
                            'Resource Development', 'Reconnaissance']
        all_tactics = techniques_df[techniques_df['ID']==row['target ID']].iloc[0]['tactics'].split(', ')
        for priority in tactics_priority:
            if priority in all_tactics:
                # 2 word tactics like "resource-development"
                technique['tactic'] = '-'.join(priority.split()).lower()
                break
        technique['metadata'] = list()
        technique['metadata'].append({'name': row['source ID'], 'value': row['description']})

        if row['source ID'] in multiplier_dict:
            technique['score'] = multiplier_dict[row['source ID']]
        else:
            technique['score'] = int(row['score'])

        technique['links'] = list()
        for label, link in row['links']:
            link_dict = dict()
            link_dict['label'] = label
            link_dict['url'] = link
            technique['links'].append(link_dict.copy())
        techniques_dict[row['target ID']] = technique.copy()

#techniques_df is everything, technique_dict is output
df.apply(df_to_layer, axis=1, techniques_dict=techniques_dict, techniques_df=techniques_df)
layer_dict['techniques'] = list(techniques_dict.values())

# default is 3 to allow minimum 3 colors set for layer
maxValue = 3
for technique in layer_dict['techniques']:
    if technique['score'] > maxValue:
        maxValue = technique['score']

# set layer background color based on high and low scores
layer_dict['gradient']['maxValue'] = maxValue

output_layer = Layer(layer_dict)
output_layer.to_file('layer.json')

[WARNING] - unrecognized navigator version 4.9.1. Defaulting to the 5.X.X schema, this may result in unexpected behavior.


-------------------------

# Import ATT&CK Navigator Layer (json)

In [18]:
filename = 'layer.json'

import_layer = Layer()
import_layer.from_file(filename)
layer_dict = import_layer.to_dict()

In [19]:
layer_df = pd.DataFrame(layer_dict['techniques'])
layer_df['Parent Technique ID'] = layer_df['techniqueID'].str.extract(r'(.{5})')

# order techniques based on sum of entire technique (including sub-techniques), within a technique, order by individual score
# for example, if a sub-technique has a low overlap score, but its overall technique score (sum) is high
# it will be prioritized first before other techniques that may individually score higher than it
order_df = layer_df.groupby(['Parent Technique ID'], as_index=False).sum(['score']).sort_values(['score'], ascending=False)
layer_df = layer_df[layer_df['score']>0].dropna().set_index('Parent Technique ID')
# order by parent technique first, then order by score
layer_df = layer_df.sort_values(by=['score'], ascending=False).loc[order_df['Parent Technique ID']]
layer_df = layer_df[['tactic', 'techniqueID', 'metadata', 'links']].copy()
layer_df = layer_df.rename(columns={'techniqueID': 'Technique ID'})

# Process Each Field into CSV

In [20]:
splunk_coverage = requests.get(splunk_coverage_url).text
import_layer = Layer()
import_layer.from_str(splunk_coverage)
coverage_dict = import_layer.to_dict()
splunk_coverage_df = pd.DataFrame(coverage_dict['techniques'])

[Version] - V3 version field detected. Upgrading to V4 Versions object.


In [21]:
splunk_raw = requests.get(splunk_raw_url)
splunk_zip = zipfile.ZipFile(io.BytesIO(splunk_raw.content))
splunk_zip.extractall(".")

In [22]:
car_url = 'https://car.mitre.org/analytics/by_technique'
car_page = requests.get(car_url)

car_coverage = requests.get(car_coverage_url).text
import_layer = Layer()
import_layer.from_str(car_coverage)
coverage_dict = import_layer.to_dict()
car_coverage_df = pd.DataFrame(coverage_dict['techniques'])

[NOTICE] - Forcibly upgrading version from 4.3 to 4.5.
[WARNING] - unrecognized navigator version 4.4. Defaulting to the 5.X.X schema, this may result in unexpected behavior.


In [23]:
# there was at least 2 I had to manually rename as of 10JUL2024

def row_format(row, techniques_df, citations_df, tools, csv=True):
    # Tactic Format
    tactic = row['tactic'].split('-')
    if len(tactic) > 1:
        for i in range(len(tactic)):
            if tactic[i].lower() not in ['and', 'or']:
                tactic[i] = tactic[i].capitalize()
        joined = ' '.join(tactic)
    else:
        joined = tactic[0].capitalize()
    row['Tactic'] = f'Has the adversary used {joined} on/in the network environment'

    # Indicator Format
    technique_name = techniques_df[techniques_df['ID']==row['Technique ID']].iloc[0]['name']
    row['Indicator'] = f'Is there evidence of {technique_name}?'

    # Evidence Format
    row['Evidence'] = ''
    evidence = []
    for metadata in row['metadata']:
        evidence.append(f"{metadata['name']}: {metadata['value']}")
    row['Evidence'] = '\n\n'.join(evidence)

    # Data Source Format
    data_sources = techniques_df[techniques_df['ID']==row['Technique ID']].iloc[0]['data sources']
    if isinstance(data_sources, str):
        row['Data Source'] = '\n\n'.join(data_sources.split(', '))

    # Description Format
    row['Description'] = techniques_df[techniques_df['ID']==row['Technique ID']].iloc[0]['description']
    ## have to use external function to do search because cannot use direct group reference (r'\1) outside of sub function context
    def get_link(matchobj):
        formatted = f" ([Citation: {matchobj.group(1)}]({citations_df[citations_df['reference'].str.contains(matchobj.group(1))].iloc[0]['url']}))"
        return formatted
    def add_link(desc):
        desc_linked = re.sub(r'\(Citation: ([^)]+)\)', get_link, desc)
        return desc_linked
    row['Description'] = add_link(row['Description'])

    # Analytic Format
    ## Cyber Analytics Repository
    analytics = []
    techniqueID = row['Technique ID']
    if sum(car_coverage_df['techniqueID']==techniqueID) > 0:
        car_url = 'https://car.mitre.org/analytics/by_technique'
        md_format = f'[Cyber Analytics Repository: {techniqueID}]({car_url})'
        analytics.append(md_format)
    ## Splunk Security Content
    if sum(splunk_coverage_df['techniqueID']==techniqueID) > 0:
        links = splunk_coverage_df.loc[splunk_coverage_df['techniqueID']==techniqueID]['comment'].item().split('\n\n')
        for link in links:
            if 'deprecated' not in link:
                md_format = f'[Splunk Security Content: {techniqueID}]({link})'
                analytics.append(md_format)
                location = f'security_content-{security_content_version}/'+re.search('detections/.*$', link).group()
                # uncomment below for debugging
                #print(location)
                if csv==False:
                    try:
                        with open(location) as yaml:
                            # add as code block
                            analytics.append(f"```yaml\n{yaml.read()}\n```")
                    except FileNotFoundError:
                        print(f'file misnamed and skipping, grab it manually or rename locally: {location}')
    ## Tool Analysis Result Sheet
    url = 'https://raw.githubusercontent.com/JPCERTCC/ToolAnalysisResultSheet/master/tool-list.html'
    for tool in tools:
        if tool in row['Evidence'].lower() or tool in row['Description'].lower():
            view_url = 'https://jpcertcc.github.io/ToolAnalysisResultSheet/'
            md_format = f'[Tool Analysis Results Sheet: {tool}]({view_url})'
            if md_format not in analytics:
                analytics.append(md_format)
    row['Analytic'] = '\n\n'.join(analytics)

    # Reference Format
    refs = []
    for ref in row['links']:
        if 'label' in ref:
            label = ref['label']
            url = ref['url']
            refs.append(f"[{label}]({url})")
    row['Reference'] = '\n\n'.join(refs)

    # Attribution Format
    row['Attribution'] = ''
    attributions = []
    for evidence in row['metadata']:
        attributions.append(evidence['name'])
    row['Attribution'] = '\n\n'.join(attributions)

    return row

layer_df['Tactic'] = ''
layer_df['Indicator'] = ''
layer_df['Data Source'] = ''
layer_df['Evidence'] = ''
layer_df['Description'] = ''
layer_df['Reference'] = ''
layer_df['Analytic'] = ''
layer_df['NAI'] = 'Not Specified'
layer_df['Attribution'] = ''

url = 'https://raw.githubusercontent.com/JPCERTCC/ToolAnalysisResultSheet/master/tool-list.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find('tbody').find_all('a', class_='nav-link')
tools = []
for result in results:
    search = re.search('(?P<tool>[^(]*)\([^)]+\)', result.text.lower())
    if search:
        tools.append(search.group('tool').rstrip())
    else:
        tools.append(result.text.lower())

final_df = layer_df.copy().apply(row_format, axis=1, techniques_df=techniques_df, citations_df=citations_df, tools=tools, csv=False)
final_csv_df = layer_df.copy().apply(row_format, axis=1, techniques_df=techniques_df, citations_df=citations_df, tools=tools, csv=True)

file misnamed and skipping, grab it manually or rename locally: security_content-4.35.0/detections/endpoint/living_off_the_land.yml


In [24]:
cols_to_keep = ['Tactic', 'Technique ID', 'Indicator', 'Analytic' ,'Evidence', 'Data Source', 'Description', 'Reference', 'Attribution']
final_df = final_df[cols_to_keep].reset_index().drop(columns='Parent Technique ID')
final_csv_df = final_csv_df[cols_to_keep].reset_index().drop(columns='Parent Technique ID')
final_csv_df.to_csv(os.path.join('.', 'analytic_plan.csv'), index = False)

# Static Web Analytic Plan

In [26]:
try:
    os.mkdir('techniques')
except OSError as error:
    print('Directory already exists')

all_count = 1
host_count = 1
network_count = 1

with open(f'all_techniques.md', 'w') as toc:
    toc.write('# All Techniques Table of Contents\n')

with open(f'host_techniques.md', 'w') as toc:
    toc.write('# Host Techniques Table of Contents\n')

with open(f'network_techniques.md', 'w') as toc:
    toc.write('# Network Techniques Table of Contents\n')

with open('search.md', 'w') as search_file:
    search_file.write('# All Techniques for Searching\n')

for i, row in final_df.iterrows():
    filename = f'techniques/{all_count:03}_{row["Technique ID"]}.md'

    with open(f'all_techniques.md', 'a') as toc:
        toc.write(f'{all_count}. [{row["Technique ID"]} {row["Indicator"]}]({filename})\n')
        all_count += 1

    if 'Network Traffic' in row['Data Source']:
        with open(f'network_techniques.md', 'a') as toc:
            toc.write(f'{network_count}. [{row["Technique ID"]} {row["Indicator"]}]({filename})\n')
            network_count += 1
    else:
        with open(f'host_techniques.md', 'a') as toc:
            toc.write(f'{host_count}. [{row["Technique ID"]} {row["Indicator"]}]({filename})\n')
            host_count += 1

    with open(filename, 'w') as t_file:
        page_text = (
            f'# {row["Technique ID"]} {row["Indicator"]}\n'
            f'## Tactic: {row["Tactic"]}\n'
            f'## Technique Description\n'
            f'{row["Description"]}\n'
            f'## Starting Analytics\n'
            f'{row["Analytic"]}\n'
            f'## Historical Usage (Evidence)\n'
            f'{row["Evidence"]}\n'
            f'## Data Sources\n'
            f'{row["Data Source"]}\n'
            f'## Attribution\n'
            f'{row["Attribution"]}\n'
            f'## References\n'
            f'{row["Reference"]}\n'
        )
        t_file.write(page_text)

    with open('search.md', 'a') as t_file:
        page_text = (
            f'# {row["Technique ID"]} {row["Indicator"]}\n'
            f'## Tactic: {row["Tactic"]}\n'
            f'## Technique Description\n'
            f'{row["Description"]}\n'
            f'## Starting Analytics\n'
            f'{row["Analytic"]}\n'
            f'## Historical Usage (Evidence)\n'
            f'{row["Evidence"]}\n'
            f'## Data Sources\n'
            f'{row["Data Source"]}\n'
            f'## Attribution\n'
            f'{row["Attribution"]}\n'
            f'## References\n'
            f'{row["Reference"]}\n'
        )
        t_file.write(page_text)

In [27]:
# create index.md and mkdocs.yml
index_text = '''# Static Analytic Plan

## Quick Links
- [Manual Search](search.md)
- [All Techniques](all_techniques.md)
- [Host Techniques](host_techniques.md)
- [Network Techniques](network_techniques.md)
'''
with open('index.md', 'w') as index:
    index.write(index_text)

# use this mkdocs.yml if you are planning on hosting the site with an actual web server (NGINX or Apache) instead of locally viewing through the browser
config_text = '''site_name: Static Analytic Plan

nav:
    - Home: 'index.md'
    - 'All Techniques': 'all_techniques.md'
    - 'Host Techniques': 'host_techniques.md'
    - 'Network Techniques': 'network_techniques.md'
    - 'Manual Search': 'search.md'

theme:
    name: mkdocs
    color_mode: dark
    user_color_mode_toggle: true
'''
with open('mkdocs.yml', 'w') as mkdocs_config:
    mkdocs_config.write(config_text)

# use this version if you are only planning on viewing locally
config_text = '''site_name: Static Analytic Plan

nav:
    - Home: 'index.md'
    - 'All Techniques': 'all_techniques.md'
    - 'Host Techniques': 'host_techniques.md'
    - 'Network Techniques': 'network_techniques.md'
    - 'Manual Search': 'search.md'

theme:
    name: mkdocs
    color_mode: dark
    user_color_mode_toggle: true

site_url: ""
use_directory_urls: false
plugins: []
'''
with open('mkdocs.yml', 'w') as mkdocs_config:
    mkdocs_config.write(config_text)

In [28]:
!python3 -m mkdocs new static_analytic_plan
!cp -r techniques static_analytic_plan/docs
!cp all_techniques.md host_techniques.md network_techniques.md search.md index.md static_analytic_plan/docs
!cp mkdocs.yml static_analytic_plan/mkdocs.yml
!cd static_analytic_plan && python3 -m mkdocs build

INFO    -  Creating project directory: static_analytic_plan
INFO    -  Writing config file: static_analytic_plan/mkdocs.yml
INFO    -  Writing initial docs: static_analytic_plan/docs/index.md
INFO    -  Cleaning site directory
INFO    -  Building documentation to directory: /content/static_analytic_plan/site
INFO    -  The following pages exist in the docs directory, but are not included in the "nav"
           configuration:
             - techniques/001_T1598.003.md
             - techniques/002_T1598.md
             - techniques/003_T1583.006.md
             - techniques/004_T1583.001.md
             - techniques/005_T1036.004.md
             - techniques/006_T1036.md
             - techniques/007_T1059.003.md
             - techniques/008_T1059.006.md
             - techniques/009_T1027.002.md
             - techniques/010_T1027.md
             - techniques/011_T1027.011.md
             - techniques/012_T1033.md
             - techniques/013_T1140.md
             - techniques/014_T

In [30]:
%%capture
!zip -r site.zip static_analytic_plan/site